In [1]:
from openAIgym.data_extractor import *
from Box2D import b2Vec2
import gym
import xgboost
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.python import debug as tf_debug

# sess = K.get_session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
# K.set_session(sess)


from sklearn.multioutput import MultiOutputRegressor

env = gym.make('LunarLander-v2')
de = DataExtractor(env,get_real_obs_for_luner_lander,RandomLunarAgent())
cache_de = CachingExtractorDecorator(de)


In [76]:
ANGLE = 4
ANGULAR_VEL = 5
ACTION = 8
sub_target = ANGULAR_VEL
inputs = range(8,12)

test_data = cache_de.get_data(1000, 10).sub_target_dataset(sub_target).sub_input_dataset(inputs)
val_data = cache_de.get_data(2000, 10).sub_target_dataset(sub_target).sub_input_dataset(inputs)
train_data = cache_de.get_data(3000, 100).sub_target_dataset(sub_target).sub_input_dataset(inputs)


In [77]:
regressor1 = xgboost.XGBRegressor(
  objective ='reg:squarederror',
  learning_rate = 0.09,
  eval_metric = 'mae',
  n_estimators = 200
)
# regressor1 = MultiOutputRegressor(regressor1)

regressor1.fit(train_data.x, train_data.y)

train_pred = regressor1.predict(train_data.x)
train_errors = train_data.y-train_pred
test_pred = regressor1.predict(test_data.x)
errors = test_data.y-test_pred
xgerr = np.mean(np.absolute(errors),axis=0)
xgerr

0.044371237037090686

In [79]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, KFold
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras_contrib.callbacks import CyclicLR

def baseline_model():
    model = Sequential()
#     model.add(Dense(20, input_dim=12, activation='relu'))
    model.add(Dense(1,input_dim=4))
    clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=0.001,
        maximal_learning_rate=0.01,
        scale_fn=lambda x: 1.,
        scale_mode="cycle",
        step_size=10
    )
    model.run_eagerly
    adam = tf.optimizers.Adam(clr)
    model.compile(loss='mean_absolute_error', optimizer=tf.optimizers.SGD(learning_rate=0.001))
    return model
  
nnRegressor = KerasRegressor(build_fn=baseline_model, batch_size=1000,
                             validation_data = (val_data.x, val_data.y),
                             verbose=True,shuffle=True
                             # callbacks = [tensorboard_callback]
                            )


nnRegressor.fit(train_data.x, train_data.y, epochs=10)
train_pred = nnRegressor.predict(train_data.x)
train_errors = train_data.y-train_pred
test_pred = nnRegressor.predict(test_data.x)
errors = test_data.y-test_pred
np.mean(np.absolute(errors),axis=0)-xgerr



Epoch 1/10
9/9 [==============================] - 0s 17ms/step - loss: 0.2863 - val_loss: 0.3027
Epoch 2/10
9/9 [==============================] - 0s 4ms/step - loss: 0.2819 - val_loss: 0.2982
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 0.2775 - val_loss: 0.2936
Epoch 4/10
9/9 [==============================] - 0s 5ms/step - loss: 0.2730 - val_loss: 0.2890
Epoch 5/10
9/9 [==============================] - 0s 5ms/step - loss: 0.2686 - val_loss: 0.2845
Epoch 6/10
9/9 [==============================] - ETA: 0s - loss: 0.271 - 0s 4ms/step - loss: 0.2642 - val_loss: 0.2799
Epoch 7/10
9/9 [==============================] - ETA: 0s - loss: 0.264 - 0s 4ms/step - loss: 0.2597 - val_loss: 0.2754
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.2553 - val_loss: 0.2708
Epoch 9/10
9/9 [==============================] - 0s 4ms/step - loss: 0.2510 - val_loss: 0.2665
Epoch 10/10
1/1 [==============================] - 0s 18ms/step


0.20439872076526577

In [60]:
nnRegressor.model.loss

'mean_absolute_error'

In [239]:
model2 = tf.keras.Sequential([
  Dense(1,input_dim=4)
])


optimizer = tf.keras.optimizers.SGD(0.0000001)
loss_object = tf.keras.losses.mean_absolute_error

model2.set_weights([np.array([[0. ],
        [0.1],
        [ 0],
        [-0.1 ]], dtype=np.float32),
 np.array([0.0], dtype=np.float32)])

loss_history = []

In [352]:
# def train_step(model, images, labels):
with tf.GradientTape(persistent=False,) as tape:
  logits = model2(train_data.x, training=True)

  # Add asserts to check the shape of the output.

  loss_value = loss_object(train_data.y, logits)

loss_history.append(loss_value.numpy().mean())
grads = tape.gradient(loss_value, model2.trainable_variables)
optimizer.apply_gradients(zip(grads, model2.trainable_variables))

# train_step(model2, train_data.x, train_data.y)

print(grads)

print(model2.get_weights())

print(np.mean(np.absolute(model2.predict(test_data.x).flatten() - test_data.y)))

print(np.mean(np.absolute(model2.predict(train_data.x).flatten() - train_data.y)))

[<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[  458.5681 ],
       [ 1397.7175 ],
       [  449.70435],
       [-1385.1566 ]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([920.8175], dtype=float32)>]
[array([[-1.0571639e-06],
       [ 9.1816381e-02],
       [-9.4230927e-07],
       [-9.1888063e-02]], dtype=float32), array([-7.364783e-05], dtype=float32)]
0.042887335925372194
0.04443056882741729


In [380]:
def get_gradient(n):
  with tf.GradientTape(persistent=True,) as tape:
    logits = model2(train_data.x[n:n+1], training=True)
    print(logits)
    print(train_data.y[n:n+1])
    loss_value = loss_object(train_data.y[n:n+1], logits)
  return tape.gradient(loss_value, model2.trainable_variables)
  

In [387]:
get_gradient(10)

tf.Tensor([[-7.459014e-05]], shape=(1, 1), dtype=float32)
[0.02422076]


[<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
 array([[-0.],
        [-0.],
        [-1.],
        [-0.]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.], dtype=float32)>]